In [1]:
# Import des modules
import pandas as pd
import numpy as np

In [2]:
# Import des données, avec formatage de la date
players = pd.read_csv("donnees_football/Player.csv")
player_attributes = pd.read_csv("donnees_football/Player_Attributes.csv", parse_dates=["date"])
teams = pd.read_csv("donnees_football/Team.csv")
matches = pd.read_csv("donnees_football/Match.csv", parse_dates=["date"])

In [3]:
"""Question 1 : Déterminez le(s) match(s) avec la plus grande différence de buts
dans le jeu de données."""

def match_max_diff_buts():
    """ Donne le(s) match(s) avec la plus grande différence de buts
    dans le jeu de données : équipes, scores, date.

    Returns
    -------
    list[str]
        Les matchs avec la plus grande différence de buts : équipes, score, date
    """

    df = matches.copy()

    # Colonne différence de buts
    df["goal_diff"] = abs(df["home_team_goal"] - df["away_team_goal"])

    # Plus grande différence de buts
    max_goal_diff = max(df["goal_diff"])

    # Matchs cherchés
    res_match = df[df["goal_diff"] == max_goal_diff]

    # Fusion avec teams pour avoir le nom des équipes
    res_match = res_match.merge(
        teams[["team_api_id", "team_long_name"]],
        left_on="home_team_api_id",
        right_on="team_api_id",
        how="left"
    ).merge(
        teams[["team_api_id", "team_long_name"]],
        left_on="away_team_api_id",
        right_on="team_api_id",
        how="left",
        suffixes=('_home', '_away')
    )

    # Résultat sous forme esthétique
    res_beau = []
    for idx, row in res_match.iterrows():
        home_name = row["team_long_name_home"]
        away_name = row["team_long_name_away"]
        home_goal = row["home_team_goal"]
        away_goal = row["away_team_goal"]
        date = row["date"].date()

        res_beau.append(f"{home_name} {home_goal} - {away_goal} {away_name} ({date})")

    return res_beau

print(match_max_diff_buts())

['PSV 10 - 0 Feyenoord (2010-10-24)']


In [4]:
"""Question 6 : Quels sont les 10 joueurs ayant obtenu la
meilleure note (overall_rating) en 2015 (meilleure note de l’année
s’il y en a plusieurs) ?"""

def meilleurs_joueurs(annee, n_joueurs):
        """
        Donne la liste des joueurs ayant obtenu la meilleure note (overall_rating)
        pour une certaine année

        Parameters
        ----------
        annee : int
                L'année considérée
        n_joueurs : int
                Le nombre de joueurs à afficher dans le top

        Returns
        -------
        list[str]
                Les meilleurs joueurs
        """
        assert isinstance(n_joueurs, int)
        assert isinstance(annee, int)

        # annee en str pour pouvoir la joindre à un str
        annee = str(annee)

        df = player_attributes.copy()

        # Nettoyage
        df = df.dropna(subset=["player_api_id", "date", "overall_rating"])

        # On ne considère que l'année annee
        df_annee = df[
                (df["date"] >= (annee + "-01-01")) &
                (df["date"] < (annee + "-12-31"))
        ]

        # Il y a plusieurs enregistrements d'attributs pour un joueur par an
        # Pour chaque joueur, on ne garde que la meilleure note, et un seul enregistrement
        df_meilleurs_annee = df_annee.groupby("player_api_id")["overall_rating"].max()
        df_annee = pd.merge(df_annee, df_meilleurs_annee, on=["player_api_id", "overall_rating"])
        df_annee = df_annee.drop_duplicates(subset="player_api_id", keep="first")

        # Classement par note descendante
        df_annee = df_annee.sort_values("overall_rating", ascending=False)

        # Fusion avec informations sur les joueurs pour avoir leurs noms
        meilleurs_joueurs_annee = pd.merge(df_annee, players, on="player_api_id")

        # Top n_joueurs
        return meilleurs_joueurs_annee["player_name"][:n_joueurs].tolist()

print(meilleurs_joueurs(2015, 10))

['Lionel Messi', 'Cristiano Ronaldo', 'Arjen Robben', 'Luis Suarez', 'Manuel Neuer', 'Eden Hazard', 'Neymar', 'Zlatan Ibrahimovic', 'Bastian Schweinsteiger', 'David Silva']


In [5]:
"""Question 7 : Déterminez le(s) match(s) avec la plus grande différence entre
les moyennes des notes des joueurs (overall_rating) des deux équipes."""

def match_max_diff_note():
    """Donne le(s) match(s) avec la plus grande différence entre
    les moyennes des notes des joueurs (overall_rating) des deux équipes.

    Returns
    -------
    list[str]
        Les matchs avec la plus grande différence entre les moyennes des notes
        des joueurs des deux équipes : équipes, notse, score, date, différence des notes
    """

    df = matches.copy()

    # On prépare l'historique des notes par joueur
    pa = player_attributes.copy()
    pa.sort_values(["player_api_id", "date"], inplace=True)

    # Dictionnaire : player_api_id -> liste (date, overall_rating)
    rating_history = (
        pa.groupby("player_api_id")[["date", "overall_rating"]]
          .apply(lambda df: list(df.itertuples(index=False, name=None)))
          .to_dict()
    )

    def rating_before(pid, match_date):
        """Renvoie la note la plus récente avant match_date (ou None s'il n'y en a pas)"""
        hist = rating_history.get(pid, [])
        # on parcourt en sens inverse pour trouver la dernière note valide
        for date, rating in reversed(hist):
            if date < match_date:
                return rating
        return None

    # Liste des variables des 11 joueurs de chaque équipe
    home_cols = [f"home_player_{i}" for i in range(1, 12)]
    away_cols = [f"away_player_{i}" for i in range(1, 12)]

    # Pour chaque match, on calcule la moyenne des notes
    df = matches.copy()

    # Nettoyage
    df = df.dropna(subset=home_cols + away_cols)

    # Moyenne de la note d'une équipe pour un match (on ne retient que si tous les joueurs ont un rating)
    def team_rating_moy(row, player_cols):
        md = row["date"]
        ratings = []
        for pid in row[player_cols]:
            rating = rating_before(pid, md)
            if rating is None:
                return None
            ratings.append(rating)
        return np.mean(ratings)

    df["home_rating"] = df.apply(lambda r: team_rating_moy(r, home_cols), axis=1)
    df["away_rating"] = df.apply(lambda r: team_rating_moy(r, away_cols), axis=1)

    # Colonne différence des ratings
    df["rating_diff"] = abs(df["home_rating"] - df["away_rating"])
    max_diff = df["rating_diff"].max()
    res = df[df["rating_diff"] == max_diff]

    # Fusion avec teams pour avoir le nom des équipes
    res = res.merge(
            teams[["team_api_id", "team_long_name"]],
            left_on="home_team_api_id", right_on="team_api_id",
            how="left"
        ).merge(
            teams[["team_api_id", "team_long_name"]],
            left_on="away_team_api_id", right_on="team_api_id",
            how="left", suffixes=("_home", "_away")
        )

    # Résultat sous forme esthétique
    res_beau = []
    for idx, row in res.iterrows():
        home = row["team_long_name_home"]
        away = row["team_long_name_away"]
        home_goal = row["home_team_goal"]
        away_goal = row["away_team_goal"]
        date = row["date"].date()
        home_rating = row["home_rating"]
        away_rating = row["away_rating"]
        diff = row["rating_diff"]
        res_beau.append(
            f"{home} (note : {home_rating:.2f}) {home_goal} - {away_goal} {away} (note : {away_rating:.2f}) ({date}) - Différence des notes : {diff:.2f}"
        )

    return res_beau

# Exemple d'appel
print(match_max_diff_note())


['FC Barcelona (note : 86.45) 3 - 0 Racing Santander (note : 67.55) (2011-10-15) - Différence des notes : 18.91']
